In [1]:
import pandas as pd
import numpy as np

from src.data_cleaning import data_labeling
from src.data_cleaning import feature_engineering
from src.language_processing import to_tfidf
from src.modeling import test_model

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
df = data_labeling('data/data_prepping_zeros_index.xlsx',
              'data/smalleyeball_partone.xlsx',
              'data/smalleyeball_parttwo.xlsx')

/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['spam_corporate'][df.screen_name.isin(combined_lst)] = 1
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['spam_bot'][df.expanded_url == 'https://acrobat.adobe.com'] = 1
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [4]:
df2 = feature_engineering(df)

/home/daniel/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df3['time_created'] = pd.to_datetime(df3['created_at']).copy()
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [6]:
df2.columns

Index(['text', 'spam', 'spam_marketing', 'spam_hijack', 'spam_corporate',
       'spam_bot', 'spam_known', 'spam_own', 'Docusign', 'onespan', 'signnow',
       'adobe sign', 'listed_count', 'description', 'statuses_count',
       'followers_count', 'favourites_count', 'friends_count',
       'time_float_sin', 'time_float_cos', 'is_description_none'],
      dtype='object')

In [8]:
text_tfidf_matrix = to_tfidf(df2,'text',100)

In [9]:
text_tfidf_matrix.shape

(20006, 100)

In [10]:
col_name_lst = []
for n in range(text_tfidf_matrix.shape[1]):
    col_name = f'text-TF-IDF-{n}'
    df2[col_name] = text_tfidf_matrix[:, n]
    col_name_lst.append(col_name)

In [23]:
data_training_set = df2[:16000]
data_test_set = df2[16000:]

In [24]:
y = data_training_set['spam_marketing']
X = data_training_set[['Docusign', 'onespan', 'signnow','adobe sign','listed_count', 'statuses_count','followers_count',
          'favourites_count', 'friends_count','time_float_sin','time_float_cos', 'is_description_none']+col_name_lst]

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
model = RandomForestClassifier()
model.fit(X_train,y_train)
y_predict = model.predict(X_test)
y_pred = model.predict_proba(X_test)

print("Accuracy score of this model is" + "\n")
print(accuracy_score(y_test,y_predict))
print("\n" + "Confusion Matrix of this model is" + "\n")
print(confusion_matrix(y_test,y_predict))
print("\n" + "Log-Loss score of this model is" + "\n")
print(log_loss(y_test,y_pred))
print("\n" + "AUC score of this model is" + "\n")
print(roc_auc_score(y_test,y_predict))

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.96425

Confusion Matrix of this model is

[[3800   23]
 [ 120   57]]

Log-Loss score of this model is

0.26847748578415215

AUC score of this model is

0.6580088403374758
